In [109]:
import numpy as np
import pandas as pd

import os
file_path = './data/covid19-global-forecasting-week-2/'
for dirname, _, filenames in os.walk(file_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/covid19-global-forecasting-week-2/test.csv
./data/covid19-global-forecasting-week-2/submission.csv
./data/covid19-global-forecasting-week-2/testing_data_with_weather_info_week_2.csv
./data/covid19-global-forecasting-week-2/training_data_with_weather_info_week_2.csv
./data/covid19-global-forecasting-week-2/train.csv


## Import data

In [116]:
train = pd.read_csv(file_path+ 'training_data_with_weather_info_week_2.csv')
test = pd.read_csv(file_path+ 'testing_data_with_weather_info_week_2.csv')
submission = pd.read_csv(file_path+ 'submission.csv')


In [117]:
train=train.drop('Id', axis=1)
train=train.rename(columns={'Id.1':'Id'})
test=test.drop('ForecastId', axis=1)
test=test.rename(columns={'ForecastId.1':'ForecastId'})

In [118]:
display(train.tail())
display(test.tail())

,Id,Country_Region,Province_State,Date,ConfirmedCases,Fatalities,country+province,Lat,Long,day_from_jan_first,temp,min,max,stp,wdsp,prcp,fog
18811,29360,Zimbabwe,NaN,2020-03-21,3.0,0.0,Zimbabwe-,-17.829167,31.052222,81,68.7,62.6,81.9,999.9,5.4,99.99,1
18812,29361,Zimbabwe,NaN,2020-03-22,3.0,0.0,Zimbabwe-,-17.829167,31.052222,82,71.4,60.8,83.7,854.2,5.3,0.00,0
18813,29362,Zimbabwe,NaN,2020-03-23,3.0,1.0,Zimbabwe-,-17.829167,31.052222,83,71.4,60.8,83.7,854.2,5.3,0.00,0
18814,29363,Zimbabwe,NaN,2020-03-24,3.0,1.0,Zimbabwe-,-17.829167,31.052222,84,71.4,60.8,83.7,854.2,5.3,0.00,0
18815,29364,Zimbabwe,NaN,2020-03-25,3.0,1.0,Zimbabwe-,-17.829167,31.052222,85,71.4,60.8,83.7,854.2,5.3,0.00,0


,ForecastId,Country_Region,Province_State,Date,country+province,day_from_jan_first,Lat,Long,temp,min,max,stp,wdsp,prcp,fog
12637,12638,Zimbabwe,NaN,2020-04-26,Zimbabwe-,51,-17.829167,31.052222,76.4,68.0,82.4,999.9,4.6,0.00,0
12638,12639,Zimbabwe,NaN,2020-04-27,Zimbabwe-,52,-17.829167,31.052222,74.3,62.4,86.2,999.9,4.0,0.00,0
12639,12640,Zimbabwe,NaN,2020-04-28,Zimbabwe-,53,-17.829167,31.052222,73.8,63.0,84.2,999.9,4.4,0.00,0
12640,12641,Zimbabwe,NaN,2020-04-29,Zimbabwe-,54,-17.829167,31.052222,75.4,67.5,82.4,999.9,6.8,0.00,1
12641,12642,Zimbabwe,NaN,2020-04-30,Zimbabwe-,55,-17.829167,31.052222,63.3,60.8,68.0,999.9,5.6,99.99,1


## Data Cleaning

In [119]:
# Format data
def Date_formatting(df):
    # Date col format
    df['Date'] = df['Date'].apply(lambda x: x.replace
                                 ('-',''))
    df['Date'] = df['Date'].astype(int)
    
    # Drop Country_Region and Province_State
        # == country+province
    df=df.drop('Country_Region', axis=1)
    df=df.drop('Province_State', axis=1)
    # '-' replacing to '_'
    df['country+province'] = df['country+province'].applt(lambda x: x.replace
                                                         ('-','_'))
    
    display(df.head())
    # print(df.isnull().sum())
    return df

In [120]:
train = Date_formatting(train)
test = Date_formatting(test)

,Id,Date,ConfirmedCases,Fatalities,country+province,Lat,Long,day_from_jan_first,temp,min,max,stp,wdsp,prcp,fog
0,1,20200122,0.0,0.0,Afghanistan-,33.0,65.0,22,42.6,33.6,54.9,999.9,9.4,0.00,0
1,2,20200123,0.0,0.0,Afghanistan-,33.0,65.0,23,42.0,32.7,55.9,999.9,14.9,99.99,1
2,3,20200124,0.0,0.0,Afghanistan-,33.0,65.0,24,40.1,36.9,43.2,999.9,10.4,0.17,1
3,4,20200125,0.0,0.0,Afghanistan-,33.0,65.0,25,46.0,37.9,56.3,999.9,6.1,0.57,1
4,5,20200126,0.0,0.0,Afghanistan-,33.0,65.0,26,42.8,36.1,53.1,999.9,10.8,0.00,1


,ForecastId,Date,country+province,day_from_jan_first,Lat,Long,temp,min,max,stp,wdsp,prcp,fog
0,1,20200319,Afghanistan-,22,33.0,65.0,42.6,33.6,54.9,999.9,9.4,0.00,0
1,2,20200320,Afghanistan-,23,33.0,65.0,42.0,32.7,55.9,999.9,14.9,99.99,1
2,3,20200321,Afghanistan-,24,33.0,65.0,40.1,36.9,43.2,999.9,10.4,0.17,1
3,4,20200322,Afghanistan-,25,33.0,65.0,46.0,37.9,56.3,999.9,6.1,0.57,1
4,5,20200323,Afghanistan-,26,33.0,65.0,42.8,36.1,53.1,999.9,10.8,0.00,1


In [123]:
train.isnull().sum()

Id                    0
Date                  0
ConfirmedCases        0
Fatalities            0
country+province      0
Lat                   0
Long                  0
day_from_jan_first    0
temp                  0
min                   0
max                   0
stp                   0
wdsp                  0
prcp                  0
fog                   0
dtype: int64

In [144]:
test.isnull().sum()

ForecastId            0
Date                  0
country+province      0
day_from_jan_first    0
Lat                   0
Long                  0
temp                  0
min                   0
max                   0
stp                   0
wdsp                  0
prcp                  0
fog                   0
dtype: int64

## PrePare Training

In [147]:
# get from weather dataset, attr
x_list = ['Date', 'Lat', 
          'Long','day_from_jan_first', 'temp',
          'min', 'max', 'stp', 'wdsp', 'wdsp',
          'prcp', 'fog'
         ]# 'country+province' not yet insert
x = train[x_list]
y1 = train[['ConfirmedCases']]
y2 = train[['Fatalities']]
x_test = test[x_list]

In [164]:
y1.describe()

,ConfirmedCases
count,18816.000000
mean,343.114052
std,3580.120962
min,0.000000
25%,0.000000
50%,0.000000
75%,15.000000
max,74386.000000


In [165]:

pred_y1_df.describe()

,ConfirmedCases_prediction
count,12642.000000
mean,327.657333
std,3304.583627
min,0.000000
25%,0.000000
50%,0.000000
75%,17.000000
max,67801.000000


In [148]:
# load RandomForest
from sklearn.ensemble import RandomForestClassifier
Tree_model = RandomForestClassifier(max_depth=200, 
                                    random_state=0)

### Train Confirmed Cases Tree

In [149]:
Tree_model.fit(x,y1)
pred_y1 = Tree_model.predict(x_test)
pred_y1_df = pd.DataFrame(pred_y1)
pred_y1_df.columns = ['ConfirmedCases_prediction']

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [156]:
from collections import Counter
# Counter(pred_y1_df['ConfirmedCases_prediction'])

Counter({0.0: 7474,
         1.0: 623,
         2.0: 247,
         10.0: 81,
         12.0: 37,
         23.0: 27,
         33.0: 15,
         38.0: 14,
         42.0: 12,
         51.0: 21,
         55.0: 7,
         59.0: 8,
         64.0: 4,
         70.0: 7,
         76.0: 38,
         104.0: 5,
         123.0: 9,
         146.0: 45,
         3.0: 242,
         5.0: 105,
         17.0: 30,
         19.0: 28,
         20.0: 28,
         24.0: 20,
         26.0: 22,
         37.0: 13,
         48.0: 8,
         54.0: 5,
         60.0: 7,
         74.0: 9,
         87.0: 6,
         90.0: 6,
         139.0: 4,
         230.0: 3,
         264.0: 1,
         302.0: 2,
         4.0: 150,
         8.0: 50,
         18.0: 77,
         52.0: 5,
         78.0: 5,
         84.0: 10,
         115.0: 4,
         136.0: 38,
         160.0: 2,
         235.0: 4,
         249.0: 7,
         265.0: 2,
         6.0: 80,
         9.0: 69,
         32.0: 17,
         39.0: 13,
         13.0: 52,
     

In [193]:
pred_y1_df.head()

,ConfirmedCases_prediction
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


### Train Deaths Tree

In [158]:
Tree_model.fit(x, y2)
pred_y2 = Tree_model.predict(x_test)
pred_y2_df = pd.DataFrame(pred_y2)
pred_y2_df.columns = ['Death_prediction']

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


## Prepare for Submission¶


In [168]:
sub_new = submission[["ForecastId"]]

In [169]:
sub_new

,ForecastId
0,1
1,2
2,3
3,4
4,5
...,...
12637,12638
12638,12639
12639,12640
12640,12641


In [188]:
submit = pd.concat([pred_y1_df, pred_y2_df, sub_new], axis =1)

submit["ConfirmedCases_prediction"] = submit["ConfirmedCases_prediction"].astype(int)
submit["Death_prediction"] = submit["Death_prediction"].astype(int)
submit.head()

,ConfirmedCases_prediction,Death_prediction,ForecastId
0,0,0,1
1,0,0,2
2,0,0,3
3,0,0,4
4,0,0,5


In [189]:
submit.columns = ['ConfirmedCases', 'Fatalities', 'ForecastId']
submit = submit[['ForecastId','ConfirmedCases', 'Fatalities']]


In [190]:
submit.describe()

,ForecastId,ConfirmedCases,Fatalities
count,12642.000000,12642.000000,12642.000000
mean,6321.500000,327.657333,9.948347
std,3649.575386,3304.583627,147.842126
min,1.000000,0.000000,0.000000
25%,3161.250000,0.000000,0.000000
50%,6321.500000,0.000000,0.000000
75%,9481.750000,17.000000,0.000000
max,12642.000000,67801.000000,3647.000000


In [191]:
submit

,ForecastId,ConfirmedCases,Fatalities
0,1,0,0
1,2,0,0
2,3,0,0
3,4,0,0
4,5,0,0
...,...,...,...
12637,12638,0,0
12638,12639,0,0
12639,12640,0,0
12640,12641,0,0


In [192]:
Sub = submit
Sub.to_csv('RF01.csv', index=False)